# CNN networks for classification of three different porkchop situations

The following code creates three convolutional neural networks:
1. A simple CNN that is trained from scratch
2. A pre-trained CNN where the convolutional layers from VGG-16 is loaded and used. 
3. A CNN with the architecture of that of 2. but without any pretrained and locked parameters

For simplicity and an easier way to grid-search for optimal hyperparameters, the Keras library is used.

### Imports

In [ ]:
import tensorflow as tf

from keras.models import Sequential
from keras import optimizers, callbacks
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Dense, Input, Conv2D, MaxPooling2D, Flatten, ZeroPadding2D
from keras import callbacks
import numpy as np

### Functions to make model-making more flexible

In [ ]:
## To simplify the creation of layers three functions are made
## Per standard the convolutional layers have 3x3 kernels, 0.5 dropout, and relu as activation function

def add_conv_layer(model, n_filters=32, kernelsize=(3,3), resize_dim=(224,224,3), num_layer=1, dropout=0.5):
    if (num_layer==0):
        model.add(Conv2D(n_filters, kernelsize, input_shape=resize_dim))
    else:
        model.add(Conv2D(n_filters, kernelsize))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    return model

def add_dense_layer(model, n_neurons, dropout=0.5, activation='relu'):
    model.add(Dense(n_neurons))
    model.add(Activation(activation))
    if dropout:
        model.add(Dropout(dropout))
    
    return model

def VGG_add_conv_layer(model, n_filters, reps, trainable=0):    
    for i in range(reps):
        model.add(ZeroPadding2D((1,1)))
        
        # Setting trainable = 0 freezes the conv.layers loaded from VGG16
        model.add(Conv2D(n_filters, (3, 3), activation='relu', trainable=trainable))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    return model

### Defining the datagenerators to feed into the networks

This code is loading images from the data folders. The point of using these is that they are a simple way to preprocess the images slightly, and hereby make the CNNs less sensitive to overfitting.

The following code uses an inbuilt keras functionality called ImageDataGenerator.

The code is based on the following link, where further information can be found:

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
def datagen(resize_dim, batch_size):

    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.3,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            rotation_range=30,
            channel_shift_range=15.0
            )
    
    test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    horizontal_flip=True,
                    vertical_flip=True,
                    rotation_range=90)
    
    train_generator = train_datagen.flow_from_directory(
            'data/train',  
            target_size=resize_dim[:2],  
            batch_size=batch_size,
            class_mode='categorical') 

    validation_generator = test_datagen.flow_from_directory(
            'data/validation',
            target_size=resize_dim[:2],
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator

### To plot the training pyplot is used

For making pretty graphs showing how the models train over time, a function is made to carry this out.

The following cell is purely aesthetics - and thus considered non-essential for understanding of the networks.

In [ ]:
import matplotlib.pyplot as plt

def model_graph(models, names, file_prefix):
    colors_train = ['g--', 'b--', 'r--']
    colors_test = ['g', 'b', 'r']
    
    for i in range(len(models)):
        plt.figure()
        plt.plot(models[i].fit.history['acc'], colors_train[i])
        plt.plot(models[i].fit.history['val_acc'], colors_test[i])
        plt.ylim(0,1)
        plt.yticks(np.arange(0,1.1,0.1))
        plt.legend([names[i] + ' train', names[i] + ' test'], loc=4)
        plt.title(names[i] + " performance", fontname="Times New Roman", fontsize=20)

        plt.savefig("graphs/"+ file_prefix + "_" + names[i] + "_acc.png", dpi=200, facecolor='w', edgecolor='w',
                orientation='portrait', papertype=None, format=None,
                transparent=False, bbox_inches='tight', pad_inches=0.1,
                frameon=None)
        plt.close()
        
    plt.figure()
    namelist = []
    for i in range(len(models)):
        plt.plot(models[i].fit.history['acc'], colors_train[i])
        plt.plot(models[i].fit.history['val_acc'], colors_test[i])
        namelist.append(names[i] + ' train')
        namelist.append(names[i] + ' test')
    
    plt.ylim(0,1)
    plt.legend(namelist, loc=4)
    plt.title("Model Performances", fontname="Times New Roman", fontsize=20)

    plt.savefig("graphs/"+ file_prefix + "_" + names[i] + "_all_acc.png", dpi=200, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format=None,
            transparent=False, bbox_inches='tight', pad_inches=0.1,
            frameon=None)
    plt.close()
    
def show_graph(models, names):
    plt.figure()
    colors_train = ['g--', 'b--', 'r--']
    colors_test = ['g', 'b', 'r']
    
    namelist = []
    for i in range(len(models)):
        plt.plot(models[i].fit.history['acc'], colors_train[i])
        plt.plot(models[i].fit.history['val_acc'], colors_test[i])
        namelist.append(names[i] + ' train')
        namelist.append(names[i] + ' test')
    
    plt.ylim(0,1)
    plt.yticks(np.arange(0,1.1,0.1))
    plt.legend(namelist, loc=4)
    plt.title("Model Performances", fontname="Times New Roman", fontsize=20)
    
    plt.show()
    plt.close()

### Classes to make model-tweaking easier

The following classes are defined to simplify the creation of networks further down in the code.

By making these it is simpler to understand and easier to crossvalidate models.

There is one for a normal CNN - used for a baseline model, and one for the VGG-16 architecture models (pretrained and untrained).

In [ ]:
class CNN_model:
    
    def __init__(self, conv_layers=[128,64,128], dense_layers=[64,16], learning_rate = 0.001, batch_size = 32):
        self.model_score = 0
        self.conv_layers = conv_layers
        self.dense_layers = dense_layers
        self.lr = learning_rate
        self.batch_size = batch_size
        self.resize_dim = (224,224,3)
        self.fit = 0
        
        # Datagen init
        self.train_generator, self.validation_generator = datagen(self.resize_dim, self.batch_size)

        # Model init
        self.model = Sequential()
    
        # Conv layers
        self.model.add(ZeroPadding2D((1,1),input_shape=self.resize_dim[:2]+(3,)))
        for l in range(len(conv_layers)):
            if (conv_layers[l]):
                self.model = add_conv_layer(self.model, conv_layers[l], num_layer = l)
    
        # Flatten matrix
        self.model.add(Flatten())  
    
        # Dense layers
        for l in range(len(dense_layers)):
            if (dense_layers[l]):
                self.model = add_dense_layer(self.model, dense_layers[l], dropout=0.5)

        # Adding the last layer with three options (due to three classes)
        self.model.add(Dense(3)) ### Fra 1 til 3
        self.model.add(Activation('sigmoid'))
    
        # Defining the optimizer used for backprobagation
        adam_op = optimizers.Adam(lr=self.lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        
        # Compiling the model
        self.model.compile(loss='categorical_crossentropy',
                      optimizer=adam_op,
                      metrics=['accuracy'])
        
        print("Model was created succesfully\n")  
    
    def train_model(self, batch_size = 16, n_epochs=50, score_batch=16, verbose=2):
        self.fit = self.model.fit_generator(
                    self.train_generator,
                    steps_per_epoch=1000 // batch_size,
                    epochs=n_epochs,
                    validation_data=self.validation_generator,
                    validation_steps=400 // batch_size,
                    verbose = verbose)
        print("Training Done")
        
    def validate_model(self, score_batch=64):
        self.model_score = self.model.evaluate_generator(self.validation_generator, score_batch)
        print("Validation Done")
        return self.model_score[1]

class CNN_model_VGG16:
    
    def __init__(self, weights_path='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', dense_layers=[16,8], learning_rate = 0.001, batch_size = 32):
        self.lr = learning_rate
        self.batch_size = batch_size
        self.resize_dim = (224,224,3)
        self.model_score = 0
        self.fit = 0
        
        # Datagen init
        self.train_generator, self.validation_generator = datagen(self.resize_dim, self.batch_size)
        
        # Model init
        self.model = Sequential()
        
        # Making the predefined VGG-16 CNN architecture
        self.model.add(ZeroPadding2D((1,1),input_shape=self.resize_dim[:2]+(3,)))
        
        self.model = VGG_add_conv_layer(self.model, 64, 2)
        
        self.model = VGG_add_conv_layer(self.model, 128, 2)
        
        self.model = VGG_add_conv_layer(self.model, 256, 3)
        
        self.model = VGG_add_conv_layer(self.model, 512, 3)
        
        self.model = VGG_add_conv_layer(self.model, 512, 2)
        
        self.model = VGG_add_conv_layer(self.model, 512, 1, trainable=True)
        
        self.model.add(Flatten())
        
        # If a path is specified, load the pre-trained weights into the architecture        
        if weights_path:
            self.model.load_weights(weights_path)
        
        # Adding the specified dense-layer architecture to the end of the VGG-16 Conv.layers.
        for i in range(len(dense_layers)):
            self.model.add(Dense(dense_layers[i]))
            self.model.add(Activation('relu'))
            self.model.add(Dropout(0.5))
        
        # Adding the last layer with three options (due to three classes)
        self.model.add(Dense(3))
        self.model.add(Activation('sigmoid'))
    
        # Defining the optimizer used for backprobagation
        adam_op = optimizers.Adam(lr=self.lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        
        # Compiling the model
        self.model.compile(loss='categorical_crossentropy',
                      optimizer=adam_op,
                      metrics=['accuracy'])
        
        print("Model was created succesfully\n")  
    
    def train_model(self, batch_size = 16, n_epochs=50, score_batch=16, verbose=2):
        self.fit = self.model.fit_generator(
                    self.train_generator,
                    steps_per_epoch=1000 // batch_size,
                    epochs=n_epochs,
                    validation_data=self.validation_generator,
                    validation_steps=400 // batch_size,
                    verbose = verbose)
        print("Training Done")

    def validate_model(self, score_batch=64):
        self.model_score = self.model.evaluate_generator(self.validation_generator, score_batch)
        print("Validation Done")
        return self.model_score[1]

## Modelling 

### Training parameters
To easily compare the different models, some training parameters are setup so that the final dense layers, the learning rate, the batch-size, and the number of epochs are the same for each model.

In [ ]:
# Training parameters (the same for both models for better comparison)
dense_end = [32] # The dense layers added to the conv.layers (both models)
adam_lr = 0.0005
b_size = 32
number_of_epochs = 100
print_amount = 0

# Training

In the following code 3 models are created and trained. To make the training flexible it is setup so that one can define how many times each type of model shall be created and trained (n_runs). Each model uses the predefined parameters defined above.

To keep track of the model performances, the final training and validation score is saved in a textfile called 00_data.txt and a graph for each created model is saved in a folder, appropriately, called Graphs.

In [ ]:
from datetime import datetime

n_runs = 3
n_trainingIms = 35
n_validationIms = 30

try:
    text_file = open('00_data.txt', 'a')
    text_file.write("\n\n")
    text_file.write("equal_set_data_(trainIms %d)(valIms %d)%s, VGG locked except last layer, epochs: %d\n"%(n_trainingIms, n_validationIms, str(dense_end), number_of_epochs))
    text_file.close()

    for i in range(n_runs):
        print("\nOwn_model training")
        baseline_model = CNN_model(conv_layers=[32,64,64], dense_layers=dense_end, learning_rate=adam_lr, batch_size=b_size)
        print("Training model...")
        baseline_model.train_model(batch_size = b_size, n_epochs=number_of_epochs, score_batch=b_size, verbose=print_amount)
        
        print("\nVGG16 training")
        VGG16_model = CNN_model_VGG16(dense_layers = dense_end, learning_rate = adam_lr, batch_size = b_size)
        print("Training model...")
        VGG16_model.train_model(batch_size = b_size, n_epochs=number_of_epochs, score_batch=b_size, verbose=print_amount)
        
        print("\nVGG16 untrained training")
        VGG16_UT_model = CNN_model_VGG16(dense_layers = dense_end, weights_path='', learning_rate = adam_lr, batch_size = b_size)
        print("Training model...")
        VGG16_UT_model.train_model(batch_size = b_size, n_epochs=number_of_epochs, score_batch=b_size, verbose=print_amount)
        
        show_graph([baseline_model, VGG16_model, VGG16_UT_model], ["Baseline", "VGG16 Pretrained", "VGG16 Untrained"])
        model_graph([baseline_model, VGG16_model, VGG16_UT_model], ["Baseline", "VGG16 Pretrained", "VGG16 Untrained"],"run_%d_lr_%.5f_%s"%(i,adam_lr,str(dense_end)))
        
        text_file = open('00_data.txt', 'a')
        
        text_file.write("Baseline: [%.4f,%.4f] \t VGG16_model: [%.4f,%.4f] \t VGG16_UT_model: [%.4f,%.4f] \t %s \n"%\
                        (baseline_model.fit.history['acc'][-1], baseline_model.fit.history['val_acc'][-1], \
                         VGG16_model.fit.history['acc'][-1], VGG16_model.fit.history['val_acc'][-1], \
                         VGG16_UT_model.fit.history['acc'][-1], VGG16_UT_model.fit.history['val_acc'][-1], \
                         str(datetime.now())))
        
        text_file.close()

        print("set %d/%d done!"%(i+1,n_runs))

    print("run done")

except KeyboardInterrupt:
    print("run done")
